### Overview

This is the first step for data preparation.

Window images, as well as reindex them 

Input: original CT images

Output: Images with abdominal windowing


In [1]:
!python -m pip install SimpleITK

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 48.4 MB 1.2 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
%reset
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import glob
import SimpleITK as sitk

import sys
os.chdir("/content/gdrive/MyDrive/One-Shot/Few-Shot-Medical-Segmentation")
py_file_location = '/content/gdrive/MyDrive/One-Shot/Few-Shot-Medical-Segmentation/dataloaders'
# sys.path.insert(0, '../../dataloaders/')
sys.path.append(os.path.abspath(py_file_location))
import niftiio as nio

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [4]:
# set up directories for images
IMG_FOLDER="/content/gdrive/MyDrive/One-Shot/Few-Shot-Medical-Segmentation/data/SABS/CT/img/"
SEG_FOLDER="/content/gdrive/MyDrive/One-Shot/Few-Shot-Medical-Segmentation/data/SABS/CT/label/"
OUT_FOLDER="/content/gdrive/MyDrive/One-Shot/Few-Shot-Medical-Segmentation/data/SABS/tmp_normalized/"

In [5]:
imgs = glob.glob(IMG_FOLDER + "/*.nii.gz")
imgs = [ fid for fid in sorted(imgs) ]
segs = [ fid for fid in sorted(glob.glob(SEG_FOLDER + "/*.nii.gz")) ]

pids = [   pid.split("img0")[-1].split(".")[0] for pid in imgs]

In [6]:
# helper function
def copy_spacing_ori(src, dst):
    dst.SetSpacing(src.GetSpacing())
    dst.SetOrigin(src.GetOrigin())
    dst.SetDirection(src.GetDirection())
    return dst

In [7]:
import copy
scan_dir = OUT_FOLDER
LIR = -125
HIR = 275
os.makedirs(scan_dir, exist_ok = True)

reindex = 0
for img_fid, seg_fid, pid in zip(imgs, segs, pids):

    img_obj = sitk.ReadImage( img_fid )
    seg_obj = sitk.ReadImage( seg_fid )

    array = sitk.GetArrayFromImage(img_obj)

    array[array > HIR] = HIR
    array[array < LIR] = LIR
    
    array = (array - array.min()) / (array.max() - array.min()) * 255.0
    
    # then normalize this
    
    wined_img = sitk.GetImageFromArray(array)
    wined_img = copy_spacing_ori(img_obj, wined_img)
    
    out_img_fid = os.path.join( scan_dir, f'image_{str(reindex)}.nii.gz' )
    out_lb_fid  = os.path.join( scan_dir, f'label_{str(reindex)}.nii.gz' ) 
    
    # then save
    sitk.WriteImage(wined_img, out_img_fid, True) 
    sitk.WriteImage(seg_obj, out_lb_fid, True) 
    print("{} has been save".format(out_img_fid))
    print("{} has been save".format(out_lb_fid))
    reindex += 1


/content/gdrive/MyDrive/One-Shot/Few-Shot-Medical-Segmentation/data/SABS/tmp_normalized/image_0.nii.gz has been save
/content/gdrive/MyDrive/One-Shot/Few-Shot-Medical-Segmentation/data/SABS/tmp_normalized/label_0.nii.gz has been save
/content/gdrive/MyDrive/One-Shot/Few-Shot-Medical-Segmentation/data/SABS/tmp_normalized/image_1.nii.gz has been save
/content/gdrive/MyDrive/One-Shot/Few-Shot-Medical-Segmentation/data/SABS/tmp_normalized/label_1.nii.gz has been save
/content/gdrive/MyDrive/One-Shot/Few-Shot-Medical-Segmentation/data/SABS/tmp_normalized/image_2.nii.gz has been save
/content/gdrive/MyDrive/One-Shot/Few-Shot-Medical-Segmentation/data/SABS/tmp_normalized/label_2.nii.gz has been save
/content/gdrive/MyDrive/One-Shot/Few-Shot-Medical-Segmentation/data/SABS/tmp_normalized/image_3.nii.gz has been save
/content/gdrive/MyDrive/One-Shot/Few-Shot-Medical-Segmentation/data/SABS/tmp_normalized/label_3.nii.gz has been save
/content/gdrive/MyDrive/One-Shot/Few-Shot-Medical-Segmentation/d

[]
